# cryptography module test

## Fernet 대칭키 암호화

- 출력이 URL safe base64 포맷
- 입력은 binary  
- 키 생성도 있음  

In [197]:
import os, random
import numpy as np

In [157]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print('key=', key, 'keylen=', len(key))
f = Fernet(key)

plain = b'Hello World!'
print('plain=', plain)
enc = f.encrypt(plain)
print('enc=', enc)
dec = f.decrypt(enc)
print('dec=', dec)

key= b'_JyosZt-YukY1tElPSUkCtx1tcTwORp-bOd-pWF16Mw=' keylen= 44
plain= b'Hello World!'
enc= b'gAAAAABdmYQo4gIuXgYuMQ2zH6J1I1ReycXTZW_BtphZ2LlzGG60_kcq4d0IC5NuVCJd8EuoX4Ua6eVgLHeAhpO0Ih0QxbMRwg=='
dec= b'Hello World!'


> 랜덤 버퍼를 만들기 위한 테스트

> 임의의 수값으로 바이트 만들기. numpy array uint8 dtype


In [159]:
bytelist = np.random.randint(0, 256, 10, dtype=np.uint8)
print(bytelist)

[120 245  47 164 176 217  58 139  10 199]


In [201]:
# 랜덤 바이트 만드는 함수는 이미 os 모듈에 있음.
print(os.urandom(20))

b'A\x85\x88\xbf\xcb\xf7\x02\xe2\xc6i\xf1\x8e\xc3e\x10\x881]\xcbr'


> 형식 변환. uint8 array to bytes

In [160]:
# byte(uint8) array to binary data. 
b1 = bytelist.tobytes()
print(b1)

x = np.frombuffer(b1, dtype=np.uint8)
print(x)

if np.all(bytelist==x):
    print('same')

b'x\xf5/\xa4\xb0\xd9:\x8b\n\xc7'
[120 245  47 164 176 217  58 139  10 199]
same


bytes와 bytearray는 모두 b'aaaa'와 같이 byte의 시퀀스 자료형.
bytes와 bytearray의 차이. bytes는 변경 불가. bytearray는 변경 가능.


In [163]:
def make_randombytes(size):
    v = np.random.randint(0, 256, size, dtype=np.uint8)
#     print(v)
    b = bytes(v)
    return b

In [166]:
b1 = make_randombytes(20)
print(b1)
print(np.frombuffer(b1, dtype=np.uint8))

b'\xca\xc0f\\)\xb5_\xbf*\x1cA\x94;}\xd0O\xda\xd3\x93\xcc'
[202 192 102  92  41 181  95 191  42  28  65 148  59 125 208  79 218 211
 147 204]


In [183]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print('key=', key, 'keylen=', len(key))
f = Fernet(key)

plain = make_randombytes(1024)
enc = f.encrypt(plain)
print(enc)
dec = f.decrypt(enc)
if plain==dec:
    print('same')
    print(plain[:10], dec[:10])

key= b'BBZ4PN2SBFGj_Sh8VAEX6IrjoOrrxfU6xQk-PpfgxpQ=' keylen= 44
b'gAAAAABdmYSl3F6tIrkVkTlqwNdcgeDqrWymn4oyAFNDCRbN3I2Fe_KLK29U5U35b4EsypahV5S4w7MNc-Mde3IiKf85s9TUZTls2iT-sXRzw95QaCYKkr271A9iZOFWfgCxncnIcVDcyAShc_YlzEKu0ExujMsswYWW1plof1DscnBColXvUMS3Y1T1nbyQ_8A9eBjPpeW3pSpCWjFOpxB5BjGTekwOU1EVr2u80S8PxDz-_XZcUx338V9N_9VRuz2UdCopskxubEs6W_marOnXa8EBPSW4H84CKOPqyNoBA_86kWHoJsX6sK7a0nXyvj6MTP23RfpLp7i8-isWjkEARZaIRdk3DPyT7y0bV_jtvLc-8baL72rxOkDvny2bXTB_TQyfXo883Vb7rQZVM7joOhXcHQZ0LfK_wrivRJm2tdPV75M-aG0RUIzkvYD4JVN7YBLfSRFyXFqdzw0NN9w2lSKr2PcI6NOfTfU0hWD6vLzfldhryW6F9jmjA_wLK44L00uL6TFTrNDK8YW8S6ZTQpn-9xPQW7A8_XX91DR4StijC77pIwK1XVOekQE7AFvp6I7NjAb0tHpeBiFm2MY0PS1zWgxkd3Lour9acV6DNXaSNKAiQpiLXCh0F5aN2gy6InUZbtId9F4DepqE0cMM8_zPmRwv24dESI_xLxTFoifYKoOmP5qCo_XPACs03cXM99PSsNfKAbSLuUcea7ErgEHZ-hGoFLktIIRHWpzgEx3wEXrhAPqBMlFMZSv92ypzbBsCBQRRPxfwvfAuOZU0P3-VToZLK7Xcw9LltcHlOzRaS9plYpmzm8Hm8sIIWQCGo4jrovXEavkgwqvkIFFBEACLBX2m3eM0l9sfVpvJr0zpe8URTAvadmP6sSvrxXffIZsb-ODyoltVD8qt4MAfoW75KjSqICa1aK

## Password PBKDF2HMAC

In [184]:
import base64
import os
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

pwd = 'password'
password= pwd.encode()
print('password=', password)
salt = os.urandom(16)  # length=16, binary
print('salt len=', len(salt))
print('salt=', salt)  # random

kdf = PBKDF2HMAC(algorithm=hashes.SHA256, 
                length=32, 
                salt=salt,
                iterations=100,
                backend=default_backend())
orgkey = kdf.derive(password)
print('orgkey len=', len(orgkey), orgkey)
key = base64.urlsafe_b64encode(orgkey)
print('key=', key)
f = Fernet(key)

plain = b'Hello World!'
print('plain=', plain)
enc = f.encrypt(plain)
print('enc=', enc)
dec = f.decrypt(enc)
print('dec=', dec)

password= b'password'
salt len= 16
salt= b'\x93?\x1cW\x9a\xfa\x0c\x0b\xd5\x95x\xb9\xabY\xfa|'
orgkey len= 32 b'\xbf\xdf\xef4\xcc2-0]\x18\xa67\x7fb\x83/\xd7\xbe\xce:M x\x0e\xdf0\x81h\x05\xd0\xf5\x87'
key= b'v9_vNMwyLTBdGKY3f2KDL9e-zjpNIHgO3zCBaAXQ9Yc='
plain= b'Hello World!'
enc= b'gAAAAABdmYSr_0x0NML-914Ka8eXq_vX0QGP1tXx1wTh8z3ouv2hJ5I7JC6SILsXI7DnSNeMarZp0ubH05J0ZB332t_5-50Cjw=='
dec= b'Hello World!'


## AES 256
AES는 블록암호화로 블록의 크기는 16바이트 (128비트이다.)

In [189]:
import os
import math
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

backend = default_backend()
key = os.urandom(32)   # 16, 32
iv = os.urandom(16)    # 16

print('key len=',len(key),  key)

cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

plainstr = 'Hello World!'*7
print('plainstr len=', len(plainstr))
plain = plainstr.encode()

# make block, null padding
blocksize=16
padsize = 16*(math.ceil(len(plain)/blocksize))-len(plain)
plain += b"\0"*padsize

print('plain=', plain)
enc = encryptor.update(plain) + encryptor.finalize()
print('enc=', enc)
dec = decryptor.update(enc) + decryptor.finalize()
print('dec=', dec)
decdec = dec.decode()
print('encode=', len(decdec), decdec)

key len= 32 b'\xd3\xc7Qo\xe5NN\xb7\xa8\xf9n\x8e?^C\x89\xd7\x8f<z\rM\x80\x12p\x0e\x81H\xc5\xef-\r'
plainstr len= 84
plain= b'Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
enc= b"\xb0\xe6\xcd/B\xc0\xe7\xf5]\xc1\xa2\x04 \xa1\x93\x93\xb7\xbbe\xc8\xb7\xd9_~L\xb0+\x08\x99\x8a\x86A\x8f\x8e\xb3\xd1:\x018\x12\x9d\xf3\xb7\x1a\xeb\x9cv\xfb\x1b\xdc7\xb0{Df\x05\xa3\r\x8f\x8d\x80\x1a\x05aK\xd7\x0bB9v\xe6YN\xa7\x02R'4K\xd2o\xd9E\x9b\xb9\x10\xcb%_\xb0X\x1e\xd8\xe6p`"
dec= b'Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
encode= 96 Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!            


## SEED 128

In [193]:
import os
import math
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

backend = default_backend()
key = os.urandom(16)   # 16, 32=not yet! (SEED 256 is not support)
iv = os.urandom(16)    # 16

print('key len=',len(key),  key)

cipher = Cipher(algorithms.SEED(key), modes.CBC(iv), backend=backend)

encryptor = cipher.encryptor()
decryptor = cipher.decryptor()

plainstr = 'Hello World!'*7
print('plainstr len=', len(plainstr), plainstr)
plain = plainstr.encode()

# make block, null padding
blocksize=16
padsize = 16*(math.ceil(len(plain)/blocksize))-len(plain)
plain += b"\0"*padsize

print('plain=', plain)
enc = encryptor.update(plain) + encryptor.finalize()
print('enc=', enc)
dec = decryptor.update(enc) + decryptor.finalize()
print('dec=', dec)
decdec = dec.decode().rstrip('\0')
print('encode=', len(decdec), decdec)


key len= 16 b'Q\x87x\xa0\x90\x84 \x97\x8e\x12\xe7Ll\x11\xe3C'
plainstr len= 84 Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!
plain= b'Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
enc= b'\nx\x18\x15\xa5\x01\x819A\xdfJJ\xac\x10q\xf4\xd4\xa6U\xe5<\x8cV\xce\x9d{\xa1\xc0\xca\x07\xc2~\xd8\x12dF<\xc8\xb05d\x1f\x8f\xf6!-\xf8v\xca\xb3\x81\xeb\x97\x8d\x19"Djj\n&G\xe2\x06\xd2\xc2\xa5\x1dYI\x1cY\xe7g)\xed>\xfa\x7f\xc7\xddxd\xc3\x83\xb4\xaf\xbd\xe4L+\x9e\xf7\xd9\x16I'
dec= b'Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
encode= 84 Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!


In [192]:
'aaa\0\0'.rstrip('\0')

'aaa'